In [1]:
import gradio as gr
import xgboost as xgb
import librosa
import parselmouth
import numpy as np
import pandas as pd
from joblib import load
import nolds
import antropy as ant
from pathlib import Path

# Load the pre-trained Parkinson's prediction model
pd_model = load('../models/Custom_model.joblib')

def extract_features(audio_path):
    sound = parselmouth.Sound(audio_path)
    y, sr = librosa.load(audio_path)

    pitch = sound.to_pitch()
    fo = pitch.selected_array['frequency']
    MDVP_Fo = np.mean(fo[fo > 0])  # Mean fundamental frequency
    MDVP_Fhi = np.max(fo)           # Max fundamental frequency
    MDVP_Flo = np.min(fo[fo > 0])   # Min fundamental frequency

    harmonicity = sound.to_harmonicity()
    HNR = harmonicity.values[harmonicity.values != -200].mean()
    NHR = 1 / HNR if HNR != 0 else 0

    RPDE = ant.perm_entropy(fo, normalize=True)
    DFA = nolds.dfa(y)

    spread = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    Spread2 = spread.mean() / (sr / 2)

    D2 = nolds.corr_dim(fo, emb_dim=25)
    PPE = ant.app_entropy(fo)

    return {
        "MDVP:Fo(Hz)": MDVP_Fo,
        "MDVP:Fhi(Hz)": MDVP_Fhi,
        "MDVP:Flo(Hz)": MDVP_Flo,
        "NHR": NHR,
        "HNR": HNR,
        "RPDE": RPDE,
        "DFA": DFA,
        "Spread2": Spread2,
        "D2": D2,
        "PPE": PPE
    }

def predict_parkinsons(audio_file):
    # The audio_file will be the path to the uploaded file
    features = extract_features(audio_file)
    feature_df = pd.DataFrame([features])

    prediction = pd_model.predict(feature_df)
    prediction_text = "Signs of Parkinson's" if prediction[0] == 1 else "Healthy"
    
    return prediction_text

# Create Gradio interface
gr_interface = gr.Interface(
    fn=predict_parkinsons,  # Function to call
    inputs=gr.Audio(type="filepath"),  # Upload audio file as file path
    outputs="text",  # Output will be a text label (prediction result)
    title="Parkinson's Prediction from Voice",  # Title of the app
    description="Upload a .wav file to predict signs of Parkinson's disease based on voice features."
)

# Launch the Gradio app
gr_interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
